In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import datetime
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('PGE_SA.csv').fillna(0)
df = df.sort_index(ascending=False)
df.index = range(len(df))
df['Data'] = pd.to_datetime(df['Data'])
df = df.rename(columns={'Maks.':'Maks'})
df = df.rename(columns={'Min.':'Min'})
df.head()

,Data,Otwarcie,Zamknięcie,Maks,Min,Obrót (mln. zł),Zmiana (%)
0,2010-10-01,17.16,16.98,17.16,16.85,30.36,0.00
1,2010-10-04,16.98,17.05,17.07,16.73,47.29,0.43
2,2010-10-05,16.99,16.82,17.00,16.82,36.29,-1.38
3,2010-10-06,16.87,16.50,16.93,16.50,48.64,-1.88
4,2010-10-07,15.72,15.84,16.05,15.67,271.00,-4.00


In [6]:
dayfeature = 150
featurenum = 4*dayfeature
x = np.zeros((df.shape[0]-dayfeature, featurenum+1))
y = np.zeros((df.shape[0]-dayfeature))

for i in range(0, df.shape[0]-dayfeature):
    x[i, 0:featurenum] = np.array(df[i:i+dayfeature][['Otwarcie', 'Zamknięcie', 'Maks', 'Min']]).reshape((1, featurenum))
    x[i, featurenum] = df.iloc[i+dayfeature]['Otwarcie']
    
for i in range(0, df.shape[0]-dayfeature):
    if df.iloc[i+dayfeature]['Zamknięcie']>=df.iloc[i+dayfeature]['Otwarcie']:
        y[i]=1
    else:
        y[i]=0

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
params={'criterion':('gini','entropy'), 'max_depth':(3,5,7,9), 'min_samples_leaf':(3,5,8,10)}

rf = RandomForestClassifier()
clf = GridSearchCV(rf, param_grid=params, cv=5)
clf.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_depth': (3, 5, 7, 9),
                         'min_samples_leaf': (3, 5, 8, 10)})

In [7]:
clf.best_params_

{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 8}

In [8]:
model = RandomForestClassifier(criterion = 'gini', max_depth=7, random_state=0, min_samples_leaf=8)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=7, min_samples_leaf=8, random_state=0)

In [9]:
result = np.mean(y_test == clf.predict(x_test))
print("random forest classifier accuacy:")
print(result)

random forest classifier accuacy:
0.4859550561797753


In [12]:
accuracy_score(y_test, clf.predict(x_test))

0.4859550561797753